In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/persona-chat

/content/drive/MyDrive/persona-chat


In [ ]:
import pandas as pd

df = pd.read_csv('raw_fake_data/Dyadic_PELD.tsv', sep='\t')

new_order = [
    'Speaker_1', 'Speaker_2', 'Utterance_1', 'Utterance_2', 'Utterance_3',
    'Personality'
]
df = df[new_order]

df.to_csv('fake_dataset/fake_data.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('fake_dataset/fake_data.csv')

speaker_personality = {}

for index, row in df.iterrows():
    speaker_personality[row['Speaker_1']] = row['Personality']

print(f'Total unique speakers: {len(speaker_personality)}')
print("List of unique speakers with their Big5 personality:")
for speaker, personality in speaker_personality.items():
    print(f'{speaker}: {personality}')

Total unique speakers: 6
List of unique speakers with their Big5 personality:
Chandler: [0.648, 0.375, 0.386, 0.58, 0.477]
Joey: [0.574, 0.614, 0.297, 0.545, 0.455]
Rachel: [0.635, 0.354, 0.521, 0.552, 0.469]
Monica: [0.713, 0.457, 0.457, 0.66, 0.511]
Phoebe: [0.6, 0.48, 0.31, 0.46, 0.56]
Ross: [0.722, 0.489, 0.6, 0.533, 0.356]


In [ ]:
! pip install OpenAI
import os
import openai
os.environ["OPENAI_API_KEY"] = "sk-proj-5PYKitPTqmecHZRDiOSAT3BlbkFJxi1JeHmicWyhX07yKHhf"

In [ ]:
import json
def process_data(name, big5):
    client = openai.OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
    )
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system",
                "content": f'Generate a detailed persona for a character from the TV show "Friends". The character\'s name is {name} and their Big Five personality traits is {big5}. Begin the description with a brief introduction such as "I am [name]", and then provide a fluent, continuous 5-sentence description in the first person. Each sentence should flow into the next, detailing the character\'s traits, motivations, and typical behaviors. Ensure the persona accurately reflects the character\'s portrayal in "Friends", with adjustments based on the Big Five personality traits. This should capture the essence of the character in a way that is precise, insightful, and aligns with their on-screen depiction.'
            },
            {
                "role": "system",
                "content": f"Name: Chandler, Big5: [0.648, 0.375, 0.386, 0.58, 0.477]\n Persona: I am Chandler. I use humor as a shield and often crack jokes to lighten up any tense situation. I feel a bit awkward in relationships but am fiercely loyal and supportive of my friends. I've made a successful career shift from statistical analysis to advertising. I struggle with a complex relationship with my family, which often influences my interactions. My sarcasm is my hallmark, but it hides a sensitive side that I don't easily show.\n Name: {name}, Big5: {big5}\n Persona: "
            }
        ]

    )
    persona_description = chat_completion.choices[0].message.content
    return persona_description

persona_dic = {}

for name, big5 in speaker_personality.items():
    persona_description = process_data(name, big5)
    print(f'Name: {name}, Big5: {big5}, Persona: {persona_description}')
    persona_dic[name] = persona_description


Name: Chandler, Big5: [0.648, 0.375, 0.386, 0.58, 0.477], Persona: I am Chandler. I use humor as a shield and often crack jokes to lighten up any tense situation. I feel a bit awkward in relationships but am fiercely loyal and supportive of my friends. I've made a successful career shift from statistical analysis to advertising. I struggle with a complex relationship with my family, which often influences my interactions. My sarcasm is my hallmark, but it hides a sensitive side that I don't easily show.
Name: Joey, Big5: [0.574, 0.614, 0.297, 0.545, 0.455], Persona: I am Joey. I exude confidence and charm, always aiming to make people around me feel good. My outgoing nature and love for acting often lead me into quirky auditions and unforgettable roles. I may come across as carefree, but my loyalty to my friends is unwavering. I possess a boldness that often propels me into hilarious misadventures, especially in my pursuit of romantic relationships. Despite my occasional naivety, I hav

In [ ]:
df = pd.read_csv('fake_data.csv')
df.rename(columns={'Personality': 'Big5'}, inplace=True)
df['Persona'] = df['Speaker_1'].map(persona_dic)
df.to_csv('updated_fake_data.csv', index=False)

In [ ]:
def combine_dialogues_into_conversation(df):
    df['Conversation'] = df['Utterance_1'] + '\n' + df['Utterance_2'] + '\n' + df['Utterance_3']
    df.drop(['Utterance_1', 'Utterance_2', 'Utterance_3'], axis=1, inplace=True)
    return df

df = pd.read_csv('updated_fake_data.csv')
new_df = combine_dialogues_into_conversation(df)
print(new_df.head())
new_df.to_csv('updatedagain_fake_data.csv', index=False)

  Speaker_1        Speaker_2                                Big5  \
0  Chandler  The Interviewer  [0.648, 0.375, 0.386, 0.58, 0.477]   
1  Chandler  The Interviewer  [0.648, 0.375, 0.386, 0.58, 0.477]   
2  Chandler  The Interviewer  [0.648, 0.375, 0.386, 0.58, 0.477]   
3  Chandler  The Interviewer  [0.648, 0.375, 0.386, 0.58, 0.477]   
4  Chandler  The Interviewer  [0.648, 0.375, 0.386, 0.58, 0.477]   

                                             Persona  \
0  I am Chandler. I use humor as a shield and oft...   
1  I am Chandler. I use humor as a shield and oft...   
2  I am Chandler. I use humor as a shield and oft...   
3  I am Chandler. I use humor as a shield and oft...   
4  I am Chandler. I use humor as a shield and oft...   

                                        Conversation  
0  also I was the point person on my company s tr...  
1  That I did. That I did.\nSo let s talk a littl...  
2  My duties?  All right.\nNow you ll be heading ...  
3  I see.\nBut there ll be perhaps

In [ ]:
def merge_consecutive_dialogues(df):
    rows = []
    previous_row = None
    last_speakers_order = None

    for index, current_row in df.iterrows():
        if previous_row is not None:
            current_speakers = (current_row['Speaker_1'], current_row['Speaker_2'])
            previous_speakers = (previous_row['Speaker_1'], previous_row['Speaker_2'])
            if frozenset(current_speakers) == frozenset(previous_speakers):
                if current_speakers == last_speakers_order:
                    new_conversation_parts = current_row['Conversation'].split('\n', 1)
                    new_conversation = new_conversation_parts[1] if len(new_conversation_parts) > 1 else ''
                else:
                    new_conversation_parts = current_row['Conversation'].rsplit('\n', 1)
                    new_conversation = new_conversation_parts[-1]
                previous_row['Conversation'] += '\n' + new_conversation
                last_speakers_order = current_speakers
                continue
            else:
                last_speakers_order = None

        if previous_row is not None:
            rows.append(previous_row)

        previous_row = current_row
        last_speakers_order = (current_row['Speaker_1'], current_row['Speaker_2'])

    if previous_row is not None:
        rows.append(previous_row)

    new_df = pd.concat([pd.DataFrame([row]) for row in rows], ignore_index=True)
    return new_df

df = pd.read_csv('updatedagain_fake_data.csv')
merged_df = merge_consecutive_dialogues(df)
print(merged_df.head())
merged_df.to_csv('merged_consecutive_data.csv', index=False)

  Speaker_1        Speaker_2                                 Big5  \
0  Chandler  The Interviewer   [0.648, 0.375, 0.386, 0.58, 0.477]   
1      Joey           Rachel  [0.574, 0.614, 0.297, 0.545, 0.455]   
2  Chandler           Monica   [0.648, 0.375, 0.386, 0.58, 0.477]   
3    Phoebe         Chandler        [0.6, 0.48, 0.31, 0.46, 0.56]   
4    Rachel             Ross  [0.635, 0.354, 0.521, 0.552, 0.469]   

                                             Persona  \
0  I am Chandler. I use humor as a shield and oft...   
1  I am Joey. I exude confidence and charm, alway...   
2  I am Chandler. I use humor as a shield and oft...   
3  I am Phoebe. My free-spirited nature and uniqu...   
4  I am Rachel. I possess a charming and outgoing...   

                                        Conversation  
0  also I was the point person on my company s tr...  
1  But then who? The waitress I went out with las...  
2  Hey, Mon.\nHey-hey-hey. You wanna hear somethi...  
3  Oh my God, oh my God! Poo